In [1]:
from typing import List, Tuple

def is_complementary(clause1: set, clause2: set) -> Tuple[bool, set]:
    for literal in clause1:
        neg_literal = f"~{literal}"
        if neg_literal in clause2:
            resolved_clause = clause1.union(clause2)
            resolved_clause.discard(literal)
            resolved_clause.discard(neg_literal)
            return True, resolved_clause
        if literal.startswith("~") and literal[1:] in clause2:
            resolved_clause = clause1.union(clause2)
            resolved_clause.discard(literal)
            resolved_clause.discard(literal[1:])
            return True, resolved_clause
    return False, set()

def resolution(kb: List[set], goal: set) -> Tuple[bool, List[Tuple[set, List[int]]]]:
    proof_steps = []
    clauses = kb + [goal]
    new_clauses = []

    while True:
        clause_pairs = [
            (clauses[i], clauses[j], (i, j))
            for i in range(len(clauses))
            for j in range(i + 1, len(clauses))
        ]

        for clause1, clause2, (i, j) in clause_pairs:
            is_complementary_pair, resolved_clause = is_complementary(clause1, clause2)
            if is_complementary_pair:
                if not resolved_clause:
                    proof_steps.append((resolved_clause, [i, j]))
                    return True, proof_steps
                if resolved_clause not in clauses and resolved_clause not in new_clauses:
                    new_clauses.append(resolved_clause)
                    proof_steps.append((resolved_clause, [i, j]))

        if not new_clauses:
            return False, proof_steps

        clauses.extend(new_clauses)
        new_clauses = []

if __name__ == "__main__":
    kb = [
        {"P", "Q"},
        {"~P", "R"},
        {"~R"},
    ]

    goal = {"~Q"}

    is_proved, proof_steps = resolution(kb, goal)

    if is_proved:
        print("PROVED")
    else:
        print("CAN'T BE PROVED")

    print("\nProof Steps:")
    for i, (clause, origins) in enumerate(proof_steps):
        print(f"Step {i + 1}: {clause} (derived from clauses {origins})")


PROVED

Proof Steps:
Step 1: {'R', 'Q'} (derived from clauses [0, 1])
Step 2: {'P'} (derived from clauses [0, 3])
Step 3: {'~P'} (derived from clauses [1, 2])
Step 4: {'Q'} (derived from clauses [0, 6])
Step 5: {'R'} (derived from clauses [1, 5])
Step 6: set() (derived from clauses [5, 6])


In [3]:
from itertools import combinations

def unify(term1, term2, substitution={}):
    if substitution is None:
        return None
    elif term1 == term2:
        return substitution
    elif isinstance(term1, str) and term1.islower():
        return unify_var(term1, term2, substitution)
    elif isinstance(term2, str) and term2.islower():
        return unify_var(term2, term1, substitution)
    elif isinstance(term1, tuple) and isinstance(term2, tuple) and len(term1) == len(term2):
        return unify(term1[1:], term2[1:], unify(term1[0], term2[0], substitution))
    else:
        return None

def unify_var(variable, value, substitution):
    if variable in substitution:
        return unify(substitution[variable], value, substitution)
    elif value in substitution:
        return unify(variable, substitution[value], substitution)
    else:
        substitution[variable] = value
        return substitution

def resolve_clause(clause1, clause2):
    resolvents = []
    for term1 in clause1:
        for term2 in clause2:
            substitution = unify(term1, negate_term(term2))
            if substitution is not None:
                new_clause = (apply_substitution(clause1, substitution) | apply_substitution(clause2, substitution)) - {term1, term2}
                resolvents.append(frozenset(new_clause))
    return resolvents

def negate_term(predicate):
    return ('not', predicate) if isinstance(predicate, str) else predicate[1]

def apply_substitution(clause, substitution):
    return {apply_single_substitution(p, substitution) for p in clause}

def apply_single_substitution(predicate, substitution):
    if isinstance(predicate, str):
        return substitution.get(predicate, predicate)
    else:
        return (predicate[0],) + tuple(substitution.get(arg, arg) for arg in predicate[1:])

def resolution_proof(kb, query):
    negated_query = frozenset({negate_term(query)})
    clauses = kb | {negated_query}
    new_clauses = set()

    while True:
        for clause1, clause2 in combinations(clauses, 2):
            resolvents = resolve_clause(clause1, clause2)
            if frozenset() in resolvents:
                return True
            new_clauses.update(resolvents)
        if new_clauses.issubset(clauses):
            return False
        clauses |= new_clauses

kb = {
    frozenset({('Mother', 'Leela', 'Oshin')}),
    frozenset({('Alive', 'Leela')}),
    frozenset({('not','Mother', 'x','y')}),
    frozenset({('Parent','x','y')}),
    frozenset({('not','Parent', 'w', 'z')}),
    frozenset({('not','Alive','w','z')}),
    frozenset({('Older','w','z')}),
}

query = ('Older', 'Leela', 'Oshin')

result = resolution_proof(kb, query)
if result:
    print("Proved by resolution: Leela is older than Oshin.")
else:
    print("Cannot prove: Leela is not older than Oshin.")


Proved by resolution: Leela is older than Oshin.
